## Get Historical data of instruments

Function to extract the historical data of instruments that include: 

- Date
- Open
- High
- Low
- Close
- Volume
- Change_percent

Values of symbols or columns through which we can search are:

- For stocks: `symbol` column
- For funds: `symbol` column
- For etfs: `symbol` column
- For indices: `symbol` column
- For forex currency pairs: `name` column
- For bonds: `name` column
- For commodities: `name` column
- For certificates: `symbol` column
- For cryptos: `symbol` column

In [ ]:
# Run and install the dependencies only once.
# ! pip install requests
# ! pip install cfscrape

#### Helpers Functions defined, that will be needed for the main execution.

In [ ]:
import requests
import json
# import cfscrape

# scraper = cfscrape.create_scraper()

## Extract the json formatted data from the investing xhr request.
def _get_json_(url:str, headers={}, params={}):
    print("Fetching {}..".format(url))
    response = requests.get(
        url=url,
        headers=headers,
        params=params
    )
    if response.status_code == 200:
        json_response = json.loads(response.text)
        return json_response
    else:
        return False


## Posting to the search bar of investing
def _post_json_(url:str, headers={}, data={}):
    print("Fetching {}..".format(url))
    response = requests.post(
        url=url, 
        headers=headers,
        data=data
    )
    if response.status_code == 200:
        json_response = json.loads(response.text)
        return json_response
    else:
        return False

#### Function defined to extract the historical data of instruments

In [ ]:
## imports
import os
import csv
import random
from user_agents import USER_AGENTS

SEARCH_BAR_URL = "https://www.investing.com/search/service/SearchInnerPage"
HISTORICAL_DATA_BASE_URL = "https://api.investing.com/api/financialdata/historical/{0}"

common_headers ={
    "X-Requested-With": "XMLHttpRequest",
    "Accept": "text/html",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "domain-id": "www",
    "user-agent": random.choice(USER_AGENTS),
    # "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"
}

## Function to extract the table of OHLC
def get_historical_data(args_tuple):
    symbol, from_date, to_date = args_tuple[0].replace("/",""), args_tuple[1], args_tuple[2]
    search_params = {
        "search_text": symbol
    }
    try:
        search_bar_data = _post_json_(
            url=SEARCH_BAR_URL,
            headers=common_headers,
            data=search_params
        )
        result = search_bar_data["quotes"]
        if len(result) > 0:
            investing_code = result[0]["pairId"]
            category = result[0]["link"].split("/")[1]
            country = result[0]["flag"]
            # print(investing_code, category, country)

        ## for data scraping
        output = []
        historical_params = {
            "start-date": from_date,
            "end-date": to_date,
            "time-frame": "Daily",
            "add-missing-rows": "False"
        }
        response_data = _get_json_(
            url=HISTORICAL_DATA_BASE_URL.format(investing_code),
            headers=common_headers,
            params=historical_params
        )
        historical_data = response_data["data"]
        for item in historical_data:
            output.append({
                "Date": item["rowDate"],
                "Open": item["last_open"],
                "High": item["last_max"],
                "Low": item["last_min"],
                "Close": item["last_close"],
                "Volume": item["volume"],
                "Change": item["change_precent"]
            })
        # print(output)
        current_dir = os.getcwd()
        if os.path.exists(f"{current_dir}/historical_data_csv"):
            path = current_dir + "/" + "historical_data_csv"
            # print(path)
        else:
            os.mkdir("historical_data_csv")
            path = current_dir + "/" + "historical_data_csv"
            # print(path)
        field_names = ["Date", "Open", "High", "Low", "Close", "Volume", "Change"]
        with open("{0}/{1}-{2}-{3}.csv".format(path,symbol,category,country), "w", newline="") as csvfile:
            writer = csv.DictWriter(csvfile,fieldnames=field_names)
            writer.writeheader()
            writer.writerows(output)
        return True
    except Exception as e:
        print("{0} not found on investing website".format(symbol))
        return False
        # return symbol

# print(get_historical_data(("LART","2022-08-01","2022-08-31")))
# print(get_historical_data(("N225INV2","2022-08-01","2022-08-31")))
# print(get_historical_data(("KTPU","2022-08-01","2022-08-31"))) 
# print(get_historical_data(("NIFTYAUTO","2022-08-01","2022-08-31")))


#### Main Function to link the `investing_code` with the searched symbol and saving the output to a csv file

In [ ]:
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

def main():
    symbols = input("Enter symbols(separated by , ): ").split(",")
    from_date = input("Enter from date(yyyy-mm-dd): ")
    to_date = input("Enter to date(yyyy-mm-dd): ")
    if datetime.strptime(from_date,"%Y-%m-%d").date() > datetime.strptime(to_date,"%Y-%m-%d").date():
        print("Your from_date is greater than the to_date")
    else:
        historical_data_args = []
        [historical_data_args.append((symbol,from_date,to_date)) for symbol in symbols]
        ############ THREADING TECHNIQUE (500 Threads) #################
        with ThreadPoolExecutor() as executor:
            executor.map(get_historical_data,historical_data_args)

        # print(historical_data_args[end-1])


        ############  LOOPING TECHNIQUE ####################
        
        # for symbol in symbols:
        #     historical_data_args.append((symbol,from_date,to_date))
        # not_found_symbols = []
        # for argument in historical_data_args:
        #     result = get_historical_data(argument)
        #     if result == True:
        #         pass
        #     else:
        #         not_found_symbols.append(result)
        # print("Not found symbols: ", not_found_symbols)


## Execution or calling of main function
#### NOTE: Inputs to be splitted by a ",".

In [ ]:
main()
# Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36

## TESTING STOCKS - Symbol Column
# TS,APBR,GGAL,TXAR,PAMP,TECO2,BPAT,ALUA,BBAR,BMA,EDN,TRAN,MIRG,BHIP,AGRO,AUSO,,GAMI,EMBR3,ENBR3,ZGPS,
# CPLE6,CRDE3,CSAN3,CSMG3,CSNA3,LIQO3,CTNM4,CYRE3,DASA3,DIRR3,DTEX3,ECOR3,EEEL3,ELEK4,ELET3,ELET6,PROJ,5EO,
# CTSA3,CTSA4,CVCB3T,DOHL4,DTCY3,EALT4,EKTR4,ELEK3,EMAE4,ENGI3,ENMT3,ESTR4,FESA3,FNAM11,FSRF11KRIP,5CP,5SR,
# PBJT,PLRP,PRPF,STVK,VBIP,VKNK,BRSP,CTRT,EDPL,EFNF,FRTF,NPRF,PVNF,UNFT,BOAT,ENPS,GINX,INGM,NTRN,SMPJ,TBOR,UNPL,600758,
# 600490,600055,600203,600346,600381,600385,600418,600556,600576,600604,600610,600629,600634,600671,600682,600685,600730,
# AURA,AVER,BBYL,FNTS,BLRX,ELWS,BRAN,BRIL,BWAY,BRMG,CAMT,CAST,CDEV,CANF,CGEN,CMER,DANE,DELT,DIFI,DIMRI,OPCT,600753

### TESTING FUNDS - SYMBOL column
# 0P0000YO4P,LP65153969,0P00012AYY,0P0000NGP0,0P00007IV4,0P0000Q7CG,0P0000O85Q,LP68040230,0P0000Q7CM,0P0000O85S,0P000077YZ,LP68077432,
# 0P00014O2D,0P0000SJE0,0P000078OP,0P00007FN6,LP65014940,LP63512373,519931,0P0000KOZZ,519015,217001,0P0000G0N4,0P0000V9U3,
# 0P0000JS6H,0P0000MXR7,240002,80002,450001,0P00009RFL,0P0000SU43,0P0000WN3S,0P0000WN3V,0P0000WN3U,0P00000CAX,0P00000RJZ,0P0000N6L3

### TESTING ETF'S - SYMBOL column 
# QOZ,FEMX,USD,VGS,A200,F100,ETHI,VGAD,FLOT,QUAL,HVST,ASIA,VEU,WDMF,FTGG,FTGE,3USLG,7GX4,KUW8G,W311,KUW8G,W311,KUW8G,W311,1HSA,H4ZF

### testing Indices - symbol column 
# AXINJD,AXMAJD,AXMEJD,AXAF,AXPBJD,AXAT,AXKO,AXJO,AXGD,AXDJ,AXSJ,AXEJ,AXFJ,AXHJ,AXNJ,AXIJ,AXMJ,AXPJ,AXJR,AXTJ,AXUJ,AXMM,AORD,
# BSEMET,BSEOIL,BSEPOWER,BSEPSU,BSEREAL,NIFVIX,B50INCDN,NIFTY200,NSED,NV20,NIFTY500,NIFMDCP100,NIMDCP50,NN50,SPBSEFMCG,NIFSMCP100,NIFTY100

## TESTING BONDS - name column 
# India 10YI,ndia 30Y,India 24Y,India 19Y,India 15Y,India 14Y,India 13Y,India 12Y,India 11Y,India 9Y,India 4Y

### TESTING CERTIFICATES - symbol column
# DEHY1Y7L,DEVZ716B,DECE8NK7,DECE6M8S,DEAA0XYR,DEAA0C4K,DE768918,DEABN0EW,DEAA0KFZ,DE918607,DEABN1EJ,DEDB3WT1,DEDZ0B77,DEDB3CTQ,DEDZ0B66,DEDZ2X4Q,DE635186

### TESTING CRYPTOS - symbol column 
# BTC,ETH,BNB,USDT,ADA,DOT,XRP,UNI,LTC,LINK,THETA,USDC,BCH,XLM,LUNA

### TESTING COMMODITIES - name column 
# MCX Aluminum Mini,MCX Aluminum,MCX Copper,MCX Copper Mini,MCX Gold 1 Kg,MCX Gold Guinea,MCX Gold Mini,MCX Gold Petal,MCX Gold Petal Del,MCX Lead,MCX Lead Mini,MCX Nickel

### TESTING Forex - name column
# USD/INR,USD/EUR,USD/JPY,INR/EUR,INR/JPY

### Note: If any of the time scraper stop to get the data. Interchange or uncomment the `user-agent` key of the `common_header`. Keep on changing, whenever it stops.